In [ ]:
from tkinter import *
from tkinter import filedialog
import os
import tkinter as tk
from PIL import Image, ImageTk
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from numpy import loadtxt
from keras.models import load_model
import easyocr
from datetime import datetime

#Prepare Model RCNN
model = load_model('rcnnplatemodel.h5')
cv2.setUseOptimized(True);
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
df = pd.read_csv (r'database_plat.csv')
df1 = pd.read_csv (r'logging.csv')

def showImage():
    fln = filedialog.askopenfilename(initialdir=os.getcwd(), title="Select Image File", filetypes=(("JPG File", "*.jpg"), ("PNG file","*.png"), ("All Files", "*.*")))
    img = Image.open(fln)
    img.thumbnail((350,350))
    img = ImageTk.PhotoImage(img)
    label.configure(image=img)
    label.image = img

def processImage():
    fln = filedialog.askopenfilename(initialdir=os.getcwd(), title="Select Image File", filetypes=(("JPG File", "*.jpg"), ("PNG file","*.png"), ("All Files", "*.*")))
    img = cv2.imread(fln)
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    imout = img.copy()
    for e,result in enumerate(ssresults):
        if e < 2000:
            x,y,w,h = result
            timage = imout[y:y+h,x:x+w]
            resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
            img = np.expand_dims(resized, axis=0)
            out= model.predict(img)
            if out[0][0] > 0.98:
                crop_imout = imout[y:y+h, x:x+w]
                break
    reader = easyocr.Reader(['en'])
    result = reader.readtext(crop_imout)
    hasil = ""
    for x in range(len(result)):
        hasil = hasil + result[x][1]
    hasil.replace(" ", "")
    label2.config(text=hasil)
    cv2.imwrite("test.jpg", crop_imout)
    img = Image.open("test.jpg")
    img.thumbnail((350,350))
    img = ImageTk.PhotoImage(img)
    label1.configure(image=img)
    label1.image = img
    
    now = datetime.now()
    current_time = now.strftime("%d:%m:%Y %H:%M:%S")
    df1.loc[len(df1.index)] = [current_time, hasil]
    df1.to_csv('logging.csv', mode='a', index = False, header=None)
    
    df = pd.read_csv (r'database_plat.csv')
    found = 0
    for i in range(len(df['plat'].values)):
        if hasil == df['plat'].values[i]:
            found = 1;
    if found == 1:
        label4.config(text="plat terdaftar")
    else:
         label4.config(text="plat tidak terdaftar")
    
def ocr():
    fln = filedialog.askopenfilename(initialdir=os.getcwd(), title="Select Image File", filetypes=(("JPG File", "*.jpg"), ("PNG file","*.png"), ("All Files", "*.*")))
    img = cv2.imread(fln)
    dim = None
    (h, w) = img.shape[:2]
    r = 500 / float(h)
    dim = (int(w * r), 500)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    reader = easyocr.Reader(['en'])
    result = reader.readtext(resized)
    hasil = ""
    for x in range(len(result)):
        hasil = hasil + result[x][1]
    hasil.replace(" ", "")
    label2.config(text=hasil)
    
    
main_window = Tk()

frm = Frame(main_window)
frm.pack(side=BOTTOM, padx=15, pady = 15)

main_window.title('Deteksi Plat Nomor dengan RCNN dan EasyOCR')
label_info_1 = Label(main_window, text = "Selamat Datang pada Deteksi Plat Nomor dengan RCNN dan EasyOCR \n\n Sumber Foto")
label_info_1.pack()

label = Label(main_window, text = "Silahkan klik Masukan Gambar")
label.pack()

label_info_2 = Label(main_window, text = "Hasil Gambar")
label_info_2.pack()

label1 = Label(main_window, text = "Silahkan klik Proses Gambar")
label1.pack()

label_info_3 = Label(main_window, text = "Hasil OCR")
label_info_3.pack()

label2 = Label(main_window, text = "Silahkan klik proces OCR")
label2.pack()

label_info_4 = Label(main_window, text = "Plat Tidak Terdaftar")
label_info_4.pack()

Button(main_window, text="Masukan Foto", command=showImage).pack()
Button(main_window, text="Proses Foto", command=processImage).pack()
Button(main_window, text="Proses OCR", command=ocr).pack()
Button(main_window, text="Exit", command=lambda: exit()).pack()

main_window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Matthew Brandon Dani\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-1-59207cc5e223>", line 75, in processImage
    label4.config(text="plat tidak terdaftar")
NameError: name 'label4' is not defined
